In [20]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,confusion_matrix,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler

np.random.seed(0)

X = np.genfromtxt("data/X_train.txt", delimiter = None)
Y = np.genfromtxt("data/Y_train.txt", delimiter = None)

X_train, X_val, Y_train, Y_val = ml.splitData(X, Y, .75)
X_test = np.genfromtxt("data/X_test.txt", delimiter = None)

sc1 = StandardScaler()
sc1.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [24]:
#learner_bayes = ml.bayes.gaussClassify()
#learner_bayes.train(X_train, Y_train)
#print("train: err = {}, auc = {}".format(learner_bayes.err(X_train, Y_train), learner_bayes.auc(X_train, Y_train)))
#print("val  : err = {}, auc = {}".format(learner_bayes.err(X_val, Y_val), learner_bayes.auc(X_val, Y_val)))

alpha = np.zeros(shape = (X_train.shape[1], 1))
weights = np.ones(shape=(X_train.shape[0],))

for i in range(0,X_train.shape[1]):
    model = GaussianNB()
    model.fit(X_train[:,i].reshape(-1,1),Y_train,np.atleast_1d(weights))
    joblib.dump(model,'data/nb_model_'+str(i)+'.txt')
    predictions = model.predict(X_train[:,i].reshape(-1,1))
    error = np.dot(weights,Y_train!=predictions)
    alpha[i] = 0.5*np.log(np.absolute((1-error)/error))
    weights *= np.exp(-alpha[i]*Y_train*predictions)
    weights /= weights.sum()

In [21]:
test_data = np.genfromtxt("data/X_test.txt",delimiter = None)
test_data = sc1.transform(test_data)
#test_data = np.genfromtxt('data/transformed_test.txt',delimiter = None)
test_predictions = np.zeros(shape=(test_data.shape[0],test_data.shape[1]))
for i in range(0,14):
    model = joblib.load('data/nb_model_'+str(i)+'.txt')
    test_predictions[:,i] = model.predict_proba(test_data[:,i].reshape(-1,1))[:,1]

final_test_predictions = np.dot(test_predictions,alpha)
final_test_predictions /= sum(alpha)

In [25]:
#test_data = np.genfromtxt("data/X_test.txt",delimiter = None)
#test_data = sc1.transform(test_data)
#test_data = np.genfromtxt('data/transformed_test.txt',delimiter = None)
#test_predictions = np.zeros(shape=(test_data.shape[0],test_data.shape[1]))

val_predictions = np.zeros(shape=(X_val.shape[0], X_val.shape[1]))
for i in range(0,14):
    model = joblib.load('data/nb_model_'+str(i)+'.txt')
    val_predictions[:,i] = model.predict_proba(X_val[:,i].reshape(-1,1))[:,1]

final_val_predictions = np.dot(val_predictions,alpha)
final_val_predictions /= sum(alpha)

In [26]:
roc_auc_score(Y_val, final_val_predictions)

0.59154635079642992